Aim: classification on mbay songs: given a mbay song, can we predict its genre?

In [0]:
# Help: 
# 1) https://www.kdnuggets.com/2020/02/audio-data-analysis-deep-learning-python-part-1.html
# 2) https://www.analyticsvidhya.com/blog/2017/08/audio-voice-processing-deep-learning/

# Files: https://drive.google.com/drive/folders/0B-znYQjmJbqKTWVGUG9mQVNPZTQ

In [0]:
# loading packages

%tensorflow_version 1.x
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
import keras
from keras.models import Sequential
import warnings
from sklearn.metrics import classification_report,confusion_matrix
from numpy import argmax
import librosa.display
import IPython.display
import random
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.optimizers import SGD

warnings.filterwarnings('ignore')

In [0]:
duree = 30
genres = 'Lamson Klag Bang Baïgn'.split()

In [0]:
# Work directory

from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Waveplots and spectograms of a sample of each genre

In [0]:
pathlib.Path(f'img_data/').mkdir(parents=True, exist_ok=True)

cmap = plt.get_cmap('inferno')
plt.figure(figsize=(14,5))

for g in genres:
  filename = os.listdir(f'/content/gdrive/My Drive/Discothèque Mbaye/{g}')[0]
  songname = f'/content/gdrive/My Drive/Discothèque Mbaye/{g}/{filename}'

  x, srx = librosa.load(songname, mono=True)
  y, sr = librosa.load(songname, mono=True,offset=librosa.get_duration(x)/2, duration=duree)

  librosa.display.waveplot(y, sr=sr,offset=librosa.get_duration(x)/2);
  plt.savefig(f'img_data/wave_{g}.png')
  plt.clf()

  plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
  plt.savefig(f'img_data/spect_{g}.png')
  plt.clf()

<Figure size 1008x360 with 0 Axes>

Working on features from spectogram (stored in a csv file)

In [0]:
# # Read the songs, extract the features and write the data in a csv file

# Header for our CSV file

header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

# Extracting features from Spectrogram: Mel-frequency cepstral coefficients (MFCC), Spectral Centroid, Zero Crossing Rate, Chroma Frequencies, and Spectral Roll-off

duree = 30

file = open('dataset_'+str(duree)+'s.csv', 'w', newline='') # 'a' for openning an existing file for writing
with file:
    writer = csv.writer(file,delimiter=';')
    writer.writerow(header)

for g in genres:
    i=1
    for filename in os.listdir(f'/content/gdrive/My Drive/Discothèque Mbaye/{g}'):
        songname = f'/content/gdrive/My Drive/Discothèque Mbaye/{g}/{filename}'
        x, srx = librosa.load(songname, mono=True)
        y, sr = librosa.load(songname, mono=True,offset=librosa.get_duration(x)/2, duration=duree)
        rmse = librosa.feature.rmse(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        filename = 'chanson' + str(i)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset_'+str(duree)+'s.csv', 'a', newline='')
        with file:
            writer = csv.writer(file,delimiter=';')
            writer.writerow(to_append.split())
        i=i+1

In [0]:
# Reading the csv file

data = pd.read_csv('/content/dataset_30s.csv', sep=';')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

# Converting to numeric type
numeric = list(data.columns)
del numeric[-1]
data[numeric] = data[numeric].apply(pd.to_numeric, errors='coerce')

#data['label'].unique() # checking we have all the genres

# Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

# Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0,test_size=0.3)

print(X_train.shape); print(X_test.shape)

(113, 26)
(49, 26)


Multi-Layer Perceptron Classifier

In [0]:
# From https://www.pluralsight.com/guides/machine-learning-neural-networks-scikit-learn

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
y_pred = mlp.fit(X_train,y_train).predict(X_test)
#print(confusion_matrix(y_test,predict_test))
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.71      0.77      0.74        22
           1       0.00      0.00      0.00         0
           2       0.20      0.20      0.20         5
           3       0.84      0.73      0.78        22

    accuracy                           0.69        49
   macro avg       0.44      0.42      0.43        49
weighted avg       0.72      0.69      0.70        49



Artificial Neural Networks

In [0]:
ANN = Sequential()
ANN.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
ANN.add(layers.Dense(128, activation='relu'))
ANN.add(layers.Dense(64, activation='relu'))
ANN.add(layers.Dense(10, activation='softmax'))
ANN.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ANN.fit(X_train,y_train,epochs=100,batch_size=128)

y_pred = ANN.predict_classes(X_test)
print(classification_report(y_pred,y_test))

Epoch 1/100
113/113 [==============================] - 0s 967us/step - loss: 2.3948 - accuracy: 0.0354
Epoch 2/100
113/113 [==============================] - 0s 43us/step - loss: 2.2143 - accuracy: 0.1504
Epoch 3/100
113/113 [==============================] - 0s 39us/step - loss: 2.0550 - accuracy: 0.4336
Epoch 4/100
113/113 [==============================] - 0s 40us/step - loss: 1.9148 - accuracy: 0.6283
Epoch 5/100
113/113 [==============================] - 0s 44us/step - loss: 1.7854 - accuracy: 0.6991
Epoch 6/100
113/113 [==============================] - 0s 31us/step - loss: 1.6601 - accuracy: 0.6903
Epoch 7/100
113/113 [==============================] - 0s 37us/step - loss: 1.5382 - accuracy: 0.6903
Epoch 8/100
113/113 [==============================] - 0s 34us/step - loss: 1.4208 - accuracy: 0.7257
Epoch 9/100
113/113 [==============================] - 0s 47us/step - loss: 1.3084 - accuracy: 0.7257
Epoch 10/100
113/113 [==============================] - 0s 29us/step - loss: 1.20

Multinomial regression

In [0]:
from sklearn.linear_model import LogisticRegression

multi = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = multi.predict(X_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.71      0.68      0.69        25
           1       0.00      0.00      0.00         0
           2       0.20      0.50      0.29         2
           3       0.68      0.59      0.63        22

    accuracy                           0.63        49
   macro avg       0.40      0.44      0.40        49
weighted avg       0.68      0.63      0.65        49



K-Nearest Neighbors

In [0]:
# https://scikit-learn.org/stable/modules/neighbors.html#classification
# https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html#sphx-glr-auto-examples-neighbors-plot-classification-py

from sklearn import neighbors, datasets

n_neighbors = 10
nghbs = neighbors.KNeighborsClassifier(n_neighbors).fit(X_train, y_train)
y_pred = nghbs.predict(X_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.75      0.64      0.69        28
           1       0.00      0.00      0.00         0
           2       0.20      0.50      0.29         2
           3       0.68      0.68      0.68        19

    accuracy                           0.65        49
   macro avg       0.41      0.46      0.42        49
weighted avg       0.70      0.65      0.67        49



Support Vector Machines

In [0]:
from sklearn import svm

SVM_model = svm.SVC()#(kernel='linear')
SVM_model.fit(X_train,y_train)
y_pred = SVM_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.71      0.83      0.77        24
           1       0.00      0.00      0.00         1
           2       1.00      0.20      0.33         5
           3       0.80      0.84      0.82        19

    accuracy                           0.76        49
   macro avg       0.63      0.47      0.48        49
weighted avg       0.76      0.76      0.73        49



Random Forests

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

ForAlea = RandomForestClassifier(max_depth=2, random_state=0)
y_pred = ForAlea.fit(X_train, y_train).predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.70      0.79      0.75        24
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         5
           3       0.73      0.84      0.78        19

    accuracy                           0.71        49
   macro avg       0.36      0.41      0.38        49
weighted avg       0.63      0.71      0.67        49



Naive Bayes

In [0]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.38      0.45      0.41        20
           1       1.00      0.11      0.20         9
           2       0.20      0.20      0.20         5
           3       0.47      0.60      0.53        15

    accuracy                           0.41        49
   macro avg       0.51      0.34      0.33        49
weighted avg       0.50      0.41      0.39        49



Linear Discriminant Analysis

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

LinDis = LinearDiscriminantAnalysis()
y_pred = LinDis.fit(X_train, y_train).predict(X_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.79      0.66      0.72        29
           1       1.00      0.50      0.67         2
           2       0.20      0.50      0.29         2
           3       0.58      0.69      0.63        16

    accuracy                           0.65        49
   macro avg       0.64      0.59      0.57        49
weighted avg       0.71      0.65      0.67        49



Convolutional Neural Networks

In [0]:
# https://levelup.gitconnected.com/audio-data-analysis-using-deep-learning-with-python-part-2-4a1f40d3708d
# https://missinglink.ai/guides/convolutional-neural-networks/python-convolutional-neural-network-creating-cnn-keras-tensorflow-plain-python/

# Extracting spectograms

cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))

for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    i=1
    for filename in os.listdir(f'/content/gdrive/My Drive/Discothèque Mbaye/{g}'):
        songname = f'/content/gdrive/My Drive/Discothèque Mbaye/{g}/{filename}'
        x, srx = librosa.load(songname, mono=True)
        y, sr = librosa.load(songname, mono=True,offset=librosa.get_duration(x)/2, duration=duree)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        filename = 'chanson' + str(i)
        plt.savefig(f'img_data/{g}/{filename}.png')
        plt.clf()
        i=i+1

model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<Figure size 576x576 with 0 Axes>

In [0]:
# Split the data into the train set and test set
#!pip install split_folders

import split_folders

split_folders.ratio('/content/img_data', output="/content/data", seed=0, ratio=(.7, .3))

Copying files: 162 files [00:00, 1314.73 files/s]


In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255, # rescale all pixel values from 0-255, so after this step all our pixel values are in range (0,1)
        shear_range=0.2, #to apply some random tranfromations
        zoom_range=0.2, #to apply zoom
        horizontal_flip=True) # image will be flipped horizontally

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        '/content/data/train',
        target_size=(64, 64),
        batch_size=25,
        classes = genres,
        class_mode='categorical',
        shuffle = False)

test_set = test_datagen.flow_from_directory(
        '/content/data/val',
        target_size=(64, 64),
        batch_size=25,
        classes = genres,
        class_mode='categorical',
        shuffle = False )

Found 112 images belonging to 4 classes.
Found 50 images belonging to 4 classes.


In [0]:
model = Sequential()
input_shape=(64, 64, 3)

#1st hidden layer
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

#2nd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

#3rd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

#Flatten
model.add(Flatten())
model.add(Dropout(rate=0.5))

#Add fully connected layer.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

#Output layer
model.add(Dense(4))
model.add(Activation('softmax'))
#model.summary()

epochs = 100
batch_size = 25
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])

model.fit_generator(training_set,
        #steps_per_epoch=25,
        epochs=100 #,validation_data=test_set,validation_steps=25
        )

In [0]:
test_set.reset()
y_pred = model.predict_generator(test_set, verbose=1)

predicted_class_indices=np.argmax(y_pred,axis=1)
print(classification_report(test_set.classes,predicted_class_indices))

2/2 [==============================] - 1s 362ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00         8
           2       0.40      1.00      0.57        20
           3       0.00      0.00      0.00         3

    accuracy                           0.40        50
   macro avg       0.10      0.25      0.14        50
weighted avg       0.16      0.40      0.23        50

